## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-18-23-22-38 +0000,world,U.S.-Venezuela prisoner swap frees Americans f...,https://www.washingtonpost.com/national-securi...
1,2025-07-18-21-08-38 +0000,world,Israel says it won’t extend visa of top U.N. h...,https://www.washingtonpost.com/world/2025/07/1...
2,2025-07-18-19-19-22 +0000,world,"Bolsonaro, accused of urging U.S. hostility, o...",https://www.washingtonpost.com/world/2025/07/1...
3,2025-07-18-17-34-59 +0000,world,Why did Israel attack Syria — and what does it...,https://www.washingtonpost.com/world/2025/07/1...
4,2025-07-18-14-41-46 +0000,world,"China-linked scam centers target U.S., congres...",https://www.washingtonpost.com/national-securi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2290/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,israel,3
28,china,2
58,trump,2
1,prisoner,1
0,venezuela,1


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
8,2025-07-18-09-00-59 +0000,world,Taiwan extends drills to show China — and Trum...,https://www.washingtonpost.com/world/2025/07/1...,10
1,2025-07-18-21-08-38 +0000,world,Israel says it won’t extend visa of top U.N. h...,https://www.washingtonpost.com/world/2025/07/1...,9
6,2025-07-18-10-53-05 +0000,world,"Felix Baumgartner, skydiver who free-fell from...",https://www.washingtonpost.com/world/2025/07/1...,9
4,2025-07-18-14-41-46 +0000,world,"China-linked scam centers target U.S., congres...",https://www.washingtonpost.com/national-securi...,9
0,2025-07-18-23-22-38 +0000,world,U.S.-Venezuela prisoner swap frees Americans f...,https://www.washingtonpost.com/national-securi...,8


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
8,10,2025-07-18-09-00-59 +0000,world,Taiwan extends drills to show China — and Trum...,https://www.washingtonpost.com/world/2025/07/1...
1,9,2025-07-18-21-08-38 +0000,world,Israel says it won’t extend visa of top U.N. h...,https://www.washingtonpost.com/world/2025/07/1...
6,9,2025-07-18-10-53-05 +0000,world,"Felix Baumgartner, skydiver who free-fell from...",https://www.washingtonpost.com/world/2025/07/1...
0,8,2025-07-18-23-22-38 +0000,world,U.S.-Venezuela prisoner swap frees Americans f...,https://www.washingtonpost.com/national-securi...
2,8,2025-07-18-19-19-22 +0000,world,"Bolsonaro, accused of urging U.S. hostility, o...",https://www.washingtonpost.com/world/2025/07/1...
4,7,2025-07-18-14-41-46 +0000,world,"China-linked scam centers target U.S., congres...",https://www.washingtonpost.com/national-securi...
5,7,2025-07-18-12-15-02 +0000,world,E.U. hits Russia with new round of sanctions t...,https://www.washingtonpost.com/world/2025/07/1...
7,6,2025-07-18-10-22-12 +0000,world,How Ukraine’s allies helped Zelensky recover h...,https://www.washingtonpost.com/politics/2025/0...
3,5,2025-07-18-17-34-59 +0000,world,Why did Israel attack Syria — and what does it...,https://www.washingtonpost.com/world/2025/07/1...
9,4,2025-07-18-04-00-00 +0000,world,The rest of the world struggles to restrain Is...,https://www.washingtonpost.com/world/2025/07/1...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
